In [7]:
!pip install torch

  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/13/85/6c1092d4b06c3db1ed23d4106488750917156af0b24ab0a2d9951830b0e9/torch-2.7.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for typing-extensions>=4.10.0 from https://files.pythonhosted.org/packages/8b/54/b1ae86c0973cc6f0210b53d508ca3641fb6d0c56823f288d108bc7ab3cc8/typing_extensions-4.13.2-py3-none-any.whl.metadata
  Obtaining dependency information for sympy>=1.13.3 from https://files.pythonhosted.org/packages/a2/09/77d55d46fd61b4a135c444fc97158ef34a095e5681d0a6c10b75bf356191/sympy-1.14.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/212.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/212.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/212.5 MB 5.5 MB/s eta 0:00:39
   ---------------------------------------- 0.2/212.5 MB 2.8 MB/s eta 0:01:18
   ---------------------------------------- 0.2/212.5 MB 2.8 MB/s 

In [ ]:
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn

In [6]:
data_dir = 'dataset_Seminar5'
num_clients = 10
client_data = {}

# Load each client’s training data
for i in range(1, num_clients + 1):
    X_path = os.path.join(data_dir, f'client_datasets/client_{i}_features.csv')
    y_path = os.path.join(data_dir, f'client_datasets/client_{i}_labels.csv')
    
    X = pd.read_csv(X_path, header=None).values  # shape: (num_samples, 270)
    y = pd.read_csv(y_path, header=None).values.flatten()  # shape: (num_samples,)
    
    client_data[i] = {'X': X, 'y': y}

# Load test data
X_test = pd.read_csv(os.path.join(data_dir, 'test_features.csv'), header=None).values
y_test = pd.read_csv(os.path.join(data_dir, 'test_labels.csv'), header=None).values.flatten()

print(X.shape)  # shape: (num_samples, 270)
print(y.shape)  # shape: (num_samples,)
print(X_test.shape)  # shape: (num_samples, 270)
print(y_test.shape)  # shape: (num_samples,)

(64, 270)
(64,)
(500, 270)
(500,)


Chosen architecture: CNN (Ressim())

In [ ]:
class ResSim(nn.Module):
    """
    A simplified ResNet-style network with residual connections and sequential blocks.
    
    Adapted for CSI input (flattened to 3x30x3).

    Architecture:
        - Two residual blocks: Conv → ReLU → Conv + skip
        - Each followed by MaxPool
        - Fully connected classifier

    Args:
        num_classes (int): Number of output classes.
    """
    def __init__(self, num_classes=12): # 12 different pose classes
        super(ResSim, self).__init__()

        # Block 1
        self.block1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1)
        )
        self.shortcut1 = nn.Conv2d(3, 64, kernel_size=1)  # aligns input channels

        # Block 2
        self.block2 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1)
        )
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 1)
        self.fc = nn.Linear(64 * 7 * 3, num_classes)

    def forward(self, x):
        x = x.view(-1, 3, 30, 3)  # reshape input vector (270,) → (3, 30, 3)

        # First residual connection
        residual = self.shortcut1(x)
        x = self.block1(x)
        x = self.relu(x + residual)
        x = self.pool(x) # (64, 15, 3)

        # Second residual connection (no need for shortcut: same shape)
        residual = x
        x = self.block2(x)
        x = self.relu(x + residual)
        x = self.pool(x) # (64, 7, 3)

        x = x.view(x.size(0), -1) # flatten
        return self.fc(x)